In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date, time
import pickle
from progressbar import ProgressBar

In [2]:
#pip install progressbar
#pbar = ProgressBar()

In [3]:
df_2018 = pd.read_csv('C:/Users/Marc/git/BananaCode/data/LosAngeles/la_2018.csv')  
df_weather = pd.read_csv('C:/Users/Marc/git/BananaCode/data/LosAngeles/weather_hourly_la.csv')  
#df_2018.head(10)

In [4]:
df_2018 = df_2018.drop_duplicates(keep='last')

**Get complete LA Metro Bike 2018 Dataset**

In [5]:
df_2018_q1 = pd.read_csv('metro-bike-share-trips-2018-q1.csv')
df_2018_q2 = pd.read_csv('metro-bike-share-trips-2018-q2.csv')
df_2018_q3 = pd.read_csv('metro-bike-share-trips-2018-q3.csv')
df_2018_q4 = pd.read_csv('metro-bike-share-trips-2018-q4.csv')
df_station_table = pd.read_csv('metro-bike-share-stations-2021-10-01.csv')

df_2018_complete = df_2018_q1.append(df_2018_q2, ignore_index=False)
df_2018_complete = df_2018_complete.append(df_2018_q3, ignore_index=False)
df_2018_complete = df_2018_complete.append(df_2018_q4, ignore_index=False)

In [11]:
df_2018_complete

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,65406367,21,2018-01-01 00:04:00,2018-01-01 00:25:00,3063,34.049198,-118.252831,3018,34.043732,-118.260139,5889,0,One Way,Walk-up,NaN
1,65406366,20,2018-01-01 00:05:00,2018-01-01 00:25:00,3063,34.049198,-118.252831,3018,34.043732,-118.260139,6311,0,One Way,Walk-up,NaN
2,65406365,19,2018-01-01 00:06:00,2018-01-01 00:25:00,3063,34.049198,-118.252831,3018,34.043732,-118.260139,5753,0,One Way,Walk-up,NaN
3,65406364,22,2018-01-01 00:13:00,2018-01-01 00:35:00,3018,34.043732,-118.260139,3031,34.044701,-118.252441,6220,30,One Way,Monthly Pass,NaN
4,65406362,45,2018-01-01 00:14:00,2018-01-01 00:59:00,4204,33.988419,-118.451630,4216,34.023392,-118.479637,12436,30,One Way,Monthly Pass,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73862,112536055,9,2018-12-31 23:46:00,2018-12-31 23:55:00,3069,34.050880,-118.248253,3069,34.050880,-118.248253,6222,1,Round Trip,Walk-up,standard
73863,112536054,7,2018-12-31 23:47:00,2018-12-31 23:54:00,3069,34.050880,-118.248253,3069,34.050880,-118.248253,5730,1,Round Trip,Walk-up,standard
73864,112540414,97,2018-12-31 23:51:00,2019-01-01 01:28:00,4211,33.984928,-118.469963,4210,33.984341,-118.471550,12347,1,One Way,Walk-up,standard
73865,112537256,28,2018-12-31 23:54:00,2019-01-01 00:22:00,3005,34.048500,-118.258537,3082,34.046520,-118.237411,12128,1,One Way,Walk-up,standard


**Complete df_station_table with lat and lon for each station**

In [6]:
#use lat&lon from df_2018_complete
df_complete = df_2018_complete

#group by start station and calculate the mean (because of slightly different values in the complete datase)
df_start_geo = df_complete.groupby('start_station').mean()
df_start_geo = df_start_geo[['start_lat', 'start_lon']]
df_start_geo = df_start_geo.reset_index().rename(columns={'start_station': 'key'}).set_index('key')
df_start_geo

#group by end station
#df_end_geo = df_complete.groupby('end_station').mean()
#df_end_geo = df_end_geo[['end_lat', 'end_lon']]
#df_end_geo = df_end_geo.reset_index().rename(columns={'end_station': 'key'}).set_index('key')
#df_end_geo

df_geo = df_start_geo

df_stations_complete = df_geo.merge(df_station_table, how='left', left_on='key', right_on='3000')
df_stations_complete = df_stations_complete.rename(columns={'3000': 'station_id', 'Virtual Station': 'station_name', 'start_lat': 'lat', 'start_lon': 'lon'}).dropna()
df_stations_complete

,lat,lon,station_id,station_name,7/7/2016,N/A,Active
1,34.048500,-118.258537,3005.0,7th & Flower,7/7/2016,DTLA,Active
2,34.045540,-118.256668,3006.0,Olive & 8th,7/7/2016,DTLA,Active
3,34.050480,-118.254593,3007.0,5th & Grand,7/7/2016,DTLA,Active
4,34.046612,-118.262733,3008.0,Figueroa & 9th,7/7/2016,DTLA,Active
5,34.037048,-118.254868,3010.0,11th & Maple,7/7/2016,DTLA,Active
...,...,...,...,...,...,...,...
126,34.028679,-118.284111,4254.0,Hoover & 29th,7/9/2018,DTLA,Active
128,34.027618,-118.280678,4267.0,28th & University,11/29/2018,DTLA,Active
129,34.025860,-118.284103,4273.0,Hoover & 32nd,8/23/2018,DTLA,Active
130,34.012520,-118.285896,4275.0,Expo Park/LAFC,10/8/2018,DTLA,Active


**Only one trip at the same time**

In [7]:
##################
#IMPROVED VERSION#
##################

df_id = df_2018.sort_values(by=['bike_id','start_time'], ascending=True)
temp = df_id[['start_time','end_time', 'bike_id']]

#VEKTORISIEREN
temp = temp.assign(next_ride_start=0)
temp['next_ride_start']= temp['start_time'].shift(periods=-1)
temp = temp.assign(next_bike_id=0)
temp['next_bike_id']= temp['bike_id'].shift(periods=-1)
#VEKTORISIEREN

temp['id_compare'] = temp.apply(lambda x: True if ((x['bike_id'] == x['next_bike_id'])) else False, axis=1)
temp = temp.dropna() #drop last row because of nan
temp['check'] = temp.apply(lambda x: True if ((x['next_ride_start'] > x['end_time'])) else False, axis=1)

#Find wrong entrys in dataset (!still contains double entrys!)
df_trips_same_time = temp[(temp['check']==False)&(temp['id_compare']==True)]
print('Wrong entrys: '+str(len(df_trips_same_time)))

Wrong entrys: 6735


**Exapmle: Wrong entrys: 193634, 193636, 194167**

In [8]:
#df_trips_same_time dataframe
df_trips_same_time[(df_trips_same_time['bike_id']==5006)&(df_trips_same_time['start_time']>='2018-08-19 14:32:00')].head(10)

,start_time,end_time,bike_id,next_ride_start,next_bike_id,id_compare,check
193634,2018-08-19 14:32:00,2018-08-19 20:32:00,5006,2018-08-19 14:32:00,5006.0,True,False
193636,2018-08-19 14:32:00,2018-08-19 14:52:00,5006,2018-08-19 14:32:00,5006.0,True,False
194167,2018-08-19 20:32:00,2018-08-20 02:32:00,5006,2018-08-19 20:32:00,5006.0,True,False
212976,2018-09-07 09:45:00,2018-09-07 10:09:00,5006,2018-09-07 10:09:00,5006.0,True,False
212990,2018-09-07 10:09:00,2018-09-07 10:37:00,5006,2018-09-07 10:37:00,5006.0,True,False


In [9]:
#original dataframe
df_id[(df_id['bike_id']==5006)&(df_id['start_time']>='2018-08-19 14:32:00')].head(10)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
193634,2018-08-19 14:32:00,2018-08-19 20:32:00,4126,4126,5006,Monthly Pass,Cabrillo Beach,Cabrillo Beach
193636,2018-08-19 14:32:00,2018-08-19 14:52:00,4126,4131,5006,Monthly Pass,Cabrillo Beach,Downtown Harbor
193637,2018-08-19 14:32:00,2018-08-19 14:52:00,4126,4126,5006,Monthly Pass,Cabrillo Beach,Cabrillo Beach
194167,2018-08-19 20:32:00,2018-08-20 02:32:00,4126,4126,5006,Monthly Pass,Cabrillo Beach,Cabrillo Beach
194168,2018-08-19 20:32:00,2018-08-19 20:52:00,4126,4131,5006,Monthly Pass,Cabrillo Beach,Downtown Harbor
195828,2018-08-21 15:26:00,2018-08-21 15:58:00,4134,4131,5006,Monthly Pass,USS IOWA,Downtown Harbor
199253,2018-08-24 20:42:00,2018-08-24 21:53:00,4131,4132,5006,Walk-up,Downtown Harbor,Fanfare Fountain
199301,2018-08-24 22:12:00,2018-08-24 22:33:00,4132,4131,5006,Walk-up,Fanfare Fountain,Downtown Harbor
199754,2018-08-25 14:06:00,2018-08-25 14:15:00,4131,4133,5006,Walk-up,Downtown Harbor,Ports O'Call
199945,2018-08-25 16:11:00,2018-08-25 16:29:00,4133,4132,5006,Walk-up,Ports O'Call,Fanfare Fountain


# Notizen

In [10]:
#pickle write
#filename = 'dogs'
#outfile = open(filename,'wb')
pickle.dump(dogs_dict,outfile)
outfile.close()

NameError: name 'dogs_dict' is not defined

In [ ]:
#pickle read in
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [ ]:
#correct way to assign a new column in pandas
df_test = df_test.assign(columname=0)

In [ ]:
#Mona:
#doppelte daten --> gleiche reihen -->FUNKTION googlen
#außreiser Fahrzeit
#daten als datetime? Sekunden weg
#Marc:
#bike ID kann (zu einem Zeitpunkt) nur einmal unterwegs sein DONE
#Informationen zu den Spalten suchen: https://bikeshare.metro.net/about/data/DONE
#Stationen: Alles außerhalb raus PROBLEM: ENTFERNUNG
#Km/h Ausreißer? --> Station Entfernung??? PROBLEM: ENTFERNUNG
#Ercan:
#Start und Endtime gleich --> Raus
#Endzeitpunkt vor Startzeitpunkt -->raus
#Start Station ID muss lgeich Start Station Name sein -->End analog --> ID oder Name Falsch?